In [7]:

import tensorflow as tf
modelo = tf.keras.models.load_model('saved_model/my_model')

# Check its architecture
modelo.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 1280)              2257984   
                                                                 
 dense (Dense)               (None, 5)                 6405      
                                                                 
Total params: 2,264,389
Trainable params: 6,405
Non-trainable params: 2,257,984
_________________________________________________________________


In [26]:
from PIL import Image
import requests
from io import BytesIO
import cv2
import numpy as np

def categorizar(url):
  respuesta = requests.get(url)
  img = Image.open(BytesIO(respuesta.content))
  img = np.array(img).astype(float)/255

  img = cv2.resize(img, (224,224))
  prediccion = modelo.predict(img.reshape(-1, 224, 224, 3)) 
  for x in prediccion[0]:
        x = round(x,5)
        print(str(x))
  return np.argmax(prediccion[0], axis=-1)

def categorizar2(path):
    img = Image.open(path)
    img = np.array(img).astype(float)/255
    img = cv2.resize(img, (224,224))
    prediccion = modelo.predict(img.reshape(-1, 224, 224, 3))
    for x in prediccion[0]:
        x = '{:0.3e}'.format(x)
        print(str(x))
    return np.argmax(prediccion[0], axis=-1)

def mostrarCategoria(nombre,window_name):
    print(nombre)
    font = cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(image, nombre, (10,450), font, 3, (0, 255, 0), 2, cv2.LINE_AA)
    cv2.imshow(window_name,image)

In [9]:
#0 = carton, 1 = vidrio, 2 = metal ,3 = papel , 4 plastico
url = 'https://img2.rtve.es/a/5897383/?h=320'
prediccion = categorizar (url)
print(prediccion)

1/1 [==============================] - 1s 709ms/step
0.0
0.00274
0.0054
0.00158
0.99027
4


In [13]:
import os


datos = 'capturas'
if not os.path.exists(datos):
    print('Carpeta creada: ',datos)
    os.makedirs(datos)


In [31]:
count = 0 #para contar las capturas
capture = cv2.VideoCapture(0)
while (capture.isOpened()):
    k = cv2.waitKey(1);
    ret, frame = capture.read()
    cv2.imshow('webCam',frame)
    if (k == ord('s')):  #s para cerrar 
        break
    elif (k == ord('c')): #c para sacar captura
        cv2.imwrite(datos+'/objeto_{}.jpg'.format(count),frame)
        print('Imagen guardada:'+'/objeto_{}.jpg'.format(count))
        
        ##captura
        path = datos+'/objeto_{}.jpg'.format(count)
        resultado = categorizar2(path)
        print (resultado)
        image = cv2.imread(path)
        window_name = 'image'
        if resultado == 0:
            mostrarCategoria("carton",window_name)
        elif resultado == 1:
            mostrarCategoria("vidrio",window_name)
        elif resultado == 2:
           mostrarCategoria("metal",window_name)
        elif resultado == 3:
           mostrarCategoria("papel",window_name)
        elif resultado == 4:
            mostrarCategoria("plastico",window_name)
        count+=1
        
        


capture.release()
cv2.destroyAllWindows()


Imagen guardada:/objeto_0.jpg
1/1 [==============================] - 0s 35ms/step
6.886e-03
5.178e-03
1.288e-02
8.728e-01
1.022e-01
3
papel
